<a href="https://colab.research.google.com/github/al34n1x/DataScience/blob/master/8.Machine_Learning/09_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# Clasificación: Regresión Logística

La regresión logística nos permite resolver problemas con dos posibles estados “SI/NO”: binario o un número finito de “etiquetas” o “clases”: múltiple.
Aunque el nombre Regresión logística sugiere una operación de regresión, el objetivo de la Regresión logística es la clasificación.
 
 Algunos Ejemplos de Regresión Logística son:

* Clasificar si el correo que llega es Spam o No es Spam
* Dados unos resultados clínicos de un tumor clasificar en “Benigno” o “Maligno”
* El texto de un artículo a analizar es: Entretenimiento, Deportes, Política ó Ciencia
* A partir de historial bancario conceder un crédito o no
* Confiaremos en la implementación del paquete sklearn en Python para ponerlo en práctica.

Lo que distingue a un modelo de regresión logística del modelo de regresión lineal es que la variable de resultado en la regresión logística es binaria o dicotómica

Podemos ver en la siguiente figura que la salida de la regresión lineal pasa a través de una función de activación que puede mapear cualquier valor real entre 0 y 1.

![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/lr_ltr.png)


![alt text](https://raw.githubusercontent.com/al34n1x/DataScience/master/img/sigmoid.png)



En este notebook vamos a estudiar los conceptos básicos de Clasificación y como podemos aplicarla usando la API de scikit-learn. Nos focalizaremos en Regresión Logística, pero veremos que es muy fácil aplicar otros algoritmos con la api de scikit-learn.




Para empezar, vamos a crear un dataset sintético que podamos aplicar en nuestro primer problema de clasificación.
Será un problema de clasificación binaria en el que, para ir entendiendo los conceptos, sólo utilizaremos dos predictores.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns

In [ ]:
from sklearn.datasets import make_blobs

X, y = make_blobs(centers=2, random_state=0, n_samples=200)

print('X ~ n_samples x n_features:', X.shape)
print('y ~ n_labels:', y.shape)

print('\nFirst 5 samples:\n', X[:5, :])
print('\nFirst 5 labels:', y[:5])



Como hemos hecho que nuestros datos sean bidimiensionales, podemos mostrarlos en un plot 2D donde la primera característica corresponda al eje *x* y la segunda característica al eje *y*. 

In [ ]:
plt.scatter(X[y == 0, 0], X[y == 0, 1], s=40, label='0', marker='^')
plt.scatter(X[y == 1, 0], X[y == 1, 1], s=40, label='1', c='orange', marker='x')

plt.xlabel('first feature')
plt.ylabel('second feature')
plt.legend(loc='upper right');



Para evaluar los modelos que vamos a crear, vamos a separar el dataset en *train* y *test*. Recordemos que estos dos conjuntos nos permiten:

1. **Training**: Ajustar el modelo a los datos de entrenamiento.
2. **Test**: Evaluar la capacidad de generalizar del modelo.


Para separar los dos conjuntos utilizamos la función `train_test_split` del módulo `model_selection`. Separaremos ambos conjuntos con un ratio de 75/25.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25,
                                                    random_state=1234,
                                                    stratify=y)   # Attention: we pass the "y" as the class label

In [ ]:
X_train.shape

In [ ]:
y_train.shape



Todos los algoritmos implementados en scikit-learn están expuestos en su API mediante objetos de tipo *Estimator*, que garantiza que todos los modelos disponen de la misma API:

- `Estimator.`**`fit(X, y)`**: ajusta los parámetros del modelo a los datos
- `Estimator.`**`predict(X)`**: predice los valores de salida para datos nuevos
- `Estimator.`**`score(X, y)`**: evalúa los resultados de la predicción

Nosotros utilizaremos el objeto LogisticRegression para nuestra tarea de clasificación:

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()



Para entrenar el modelo, simplemente tenemos que llamar al método **`fit`** de nuestro objeto `classifier`, indicándole los datos de entrenamiento y las etiquetas asociadas a esos datos: 

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
classifier.coef_

In [ ]:
classifier.intercept_



Una vez tenemos el modelo entrenado, podemos empezar a realizar predicciones con datos nuevos:

In [ ]:
prediction = classifier.predict(X_test)
prediction

In [ ]:
classifier.predict_proba(X_test).round(2)

Podemos comparar visualmente las predicciones con los valores de $y$ reales:

In [ ]:
print(prediction[:20])
print(y_test[:20])



Para evaluarlo cuantitativamente, podemos computar qué fracción de las predicciones es correcta. A esta métrica se le llama **accuracy**:

In [ ]:
np.mean(prediction == y_test)



Por supuesto, scikit-learn nos provee la función **`score`** para computar el **accuracy** directamente:

In [ ]:
classifier.score(X_test, y_test)

In [ ]:
classifier.score(X_train, y_train)

In [ ]:
#dir(classifier)



Suele ser muy útil comparar la capacidad de generalización del modelo en el conjunto de test con el accuracy en los datos de entrenamiento (veremos por qué más adelante):




La Regresión Logística es un modelo linear, esto es, un modelo que crea una decisión que es lineal en el espacio de entrada. En nuestro dataset, esto significa que el umbral de decisión es una recta que separa las dos variables de entrada:

In [ ]:
#from seaborn import figures
import numpy as np
import matplotlib.pyplot as plt

plt.scatter(X[y == 0, 0], X[y == 0, 1], s=40, label='0', marker='^')
plt.scatter(X[y == 1, 0], X[y == 1, 1], s=40, label='1', c='orange', marker='x')

plt.xlabel("first feature")
plt.ylabel("second feature")
plt.legend(loc='upper right');

In [ ]:
#from seaborn import figures
import numpy as np
import matplotlib.pyplot as plt




# Plot the decision boundary. For that, we will assign a color to each
# point in the mesh [x_min, x_max]x[y_min, y_max].
x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
h = .02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
Z = classifier.predict(np.c_[xx.ravel(), yy.ravel()])
# Put the result into a color plot
Z = Z.reshape(xx.shape)
plt.figure(1, figsize=(7, 4))
plt.pcolormesh(xx, yy, Z, cmap=plt.cm.Paired)


plt.scatter(X[y == 0, 0], X[y == 0, 1], s=40, label='0', marker='^')
plt.scatter(X[y == 1, 0], X[y == 1, 1], s=40, label='1', c='orange', marker='x')

plt.xlabel("first feature")
plt.ylabel("second feature")
#mglearn.plot_2d_separator(classifier, X)
plt.legend(loc='upper right');



Podemos obtener otras métricas como F-Score, etc.

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print('Precision:', precision_score(y_test, prediction))
print('Recall:   ', recall_score(y_test, prediction))
print('Fscore:   ', f1_score(y_test, prediction))



Además es posible obtener los parámetros de la regresión:

In [ ]:
print(classifier.coef_)
print(classifier.intercept_)